# How to make tests using the eWaterCycle test framework

In this notebook it is shown how you can create a test bank and tests using the ewatercycle-model-testing system. The following topics are covered: How to create a test, the configurations a test can be in and how to make a test bank.

In [ ]:
import ewatercycle

from test_result import TestResult
from test_suite import TestSuite
from test import Test, TestType
from test_bank import TestBank
from parse_submission import ParseSubmission
import mocks
import yaml

To mark a test we use decorators, these are used as follows:

In [ ]:
@Test(name="This is the name of the test", description="this is where you explain your test", critical=True, enabled=True, test_type=TestType.BOTH)
def myFirstTest(model, discharge_name):
    print("I am testing something...")
    return TestResult(True, "I indeed made a test!")

Name can be used to give your test a custom name, if you do not specify a name the test will have the name of the test method you create.\
 Description is where you can give extra info on your test (for example what does it test). \
The boolean critical describes if a test will give a warning or an error, if false the test will give a warning, if one of these tests fail the test suite can still pass. If critical is true it will be considered an error, a hard barrier that fails the entire test suite until it is fixed.\
The enabled boolean controls if a test is enabled, this can be used for when one specific test keeps failing, and you do not want to rerun your entire test suite. \
With test_type you can specify what types of models this test should be run on, if not specified all tests will be run. 

Now lets run this test, this is all boilerplate that is done behind the scenes and changed to make the testing procedure more visual:

In [ ]:
test_suite: TestSuite = TestSuite()
model = mocks.worstModelMock()  
result = test_suite.tests["This is the name of the test"].run(model, _)
print(yaml.dump(result))

Now to make a test that actually tests something, this test will test if the .time method is implemented for a model.

In [ ]:
@Test(description="This test sees if the .time method is implemented", critical=True, enabled=True)
def myFirstCrash(model, _):
    model.time
    return TestResult(True, "time is called properly!")

Now let's run this test:

In [ ]:
myFirstTest.enabled = False
result = test_suite.tests["myFirstCrash"].run(model, _)
print(yaml.dump(result))

As you see there was an error and the test suite stopped running, this is because the worstModel does not implement the .time method and throws an error. 

To counteract these crashes we recommend you write your tests using a try-except clause as follows:

In [ ]:
@Test(description="This test sees if the .time method is implemented", critical=True, enabled=True)
def myFirstExcept(model, _):
    try:
        model.time
        return TestResult(True, "time is called properly!")
    except:
        return TestResult(False, "Test failed when calling following methods: .time")

Now let's try out this test:

In [ ]:
myFirstCrash.enabled = False
result = test_suite.tests["myFirstExcept"].run(model, _)
print(yaml.dump(result))

As you see it fails, now let's try this test on a mock that does implement time.

In [ ]:
model = mocks.BasicModelMock()
result = test_suite.tests["myFirstExcept"].run(model, _)
print(yaml.dump(result))

Now to move on to the concept of TestBanks, TestBanks can be seen as groups of tests. Similar to Tests TestBanks are denoted using decorators as such:

In [ ]:
@TestBank(description="Tests if errors are thrown when incorrect variables are called")
class ExampleBank:
    
    @staticmethod
    @Test(description="some test", critical=True, enabled=True)
    def bank_test(model, _):
        return TestResult(True, "Bank test")
    
    @staticmethod
    @Test(description="some other test", critical=True, enabled=True)
    def other_test(model, _):
        return TestResult(True, "Other test")

@TestBank(description="Tests if errors are thrown when incorrect variables are called")
class AnotherExampleBank:
    
    @staticmethod
    @Test(description="some test", critical=True, enabled=True)
    def another_bank_test(model, _):
        return TestResult(True, "another Bank test")
    
    @staticmethod
    @Test(description="some other test", critical=True, enabled=True)
    def another_other_test(model, _):
        return TestResult(True, "another Other test")
        

The test bank decorator holds a description of the test bank, use this to denote what a test bank has been designed to test.

Now to run these tests (once again this code is for demonstration of tests only, it is not run like this in actuality):

In [ ]:
myFirstExcept.enabled = False
model = mocks.BasicModelMock()
results = []
for (testbank_name, values) in test_suite.test_banks.items():
    for test in values.tests:
        if test.enabled != False: 
            results.append(test.run(model, _))
print(yaml.dump(results))

Because they are a group you can disable them as such:

In [ ]:
AnotherExampleBank.disable_all()

Now to run it on a mock version of the test system:

In [ ]:
model = mocks.BasicModelMock()
results = []
for (testbank_name, values) in test_suite.test_banks.items():
    for test in values.tests:
        if test.enabled != False: 
            results.append(test.run(model, _))
print(yaml.dump(results))

As you can see all the tests in AnotherExampleBank are no longer run, this is because they have all been disabled. You can turn the tests back on like this:

In [ ]:
AnotherExampleBank.enable_all()

As the test bank has been enabled again the tests should now run again:

In [ ]:
model = mocks.BasicModelMock()
results = []
for (testbank_name, values) in test_suite.test_banks.items():
    for test in values.tests:
        if test.enabled != False: 
            results.append(test.run(model, _))
print(yaml.dump(results))

That concludes the basics of how to make a test through the ewatercycle-model-testing system.